# ✂️ Model Pruning From Scratch: Sparsity and Significance

[!["Open In Colab"](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/adiel2012/model-size-reduction/blob/main/pruning_demo.ipynb)

## 📖 The Theory: Magnitude-based Pruning

Neural networks are often **over-parameterized**, meaning many weights contribute very little to the final result. Pruning is the process of zeroing out these insignificant weights.

### L1 Magnitude Pruning
The most common heuristic is that weights with the smallest absolute values (L1 norm) are the least important. By setting them to zero, we create a **sparse matrix**.

### Masking
In practice, we don't just delete the weight. We maintain a binary **mask** $M$ of the same shape as $W$. The effective weight used in the model is then:

$$W_{pruned} = W \odot M$$

---

In [ ]:
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel

def manual_l1_pruning(weight, amount=0.3):
    """
    Implementation of L1 Unstructured Pruning from scratch.
    weight: Tensor
    amount: Fraction of weights to prune (0.0 to 1.0)
    """
    # 1. Flatten weight and compute L1 magnitude
    w_abs = torch.abs(weight).flatten()
    
    # 2. Find the threshold value for the bottom % of weights
    k = int(amount * w_abs.numel())
    if k == 0: return weight, torch.ones_like(weight)
    
    threshold = torch.kthvalue(w_abs, k).values
    
    # 3. Create Binary Mask (1 if > threshold, 0 otherwise)
    mask = (torch.abs(weight) > threshold).float()
    
    # 4. Apply Mask
    return weight * mask, mask

# Test on GPT-2 Layer
model = GPT2LMHeadModel.from_pretrained("gpt2")
layer_weight = model.transformer.h[0].attn.c_attn.weight.data.clone()

pruned_w, mask = manual_l1_pruning(layer_weight, amount=0.5)

print(f"Original Params: {layer_weight.numel()}")
print(f"Zeroed Params:   {torch.sum(mask == 0).item()}")
print(f"Sparsity:        {100. * torch.sum(mask == 0).item() / layer_weight.numel():.2f}%")

## 🧱 Structured vs. Unstructured
While the code above prunes individual weights (**Unstructured**), modern hardware often prefers pruning entire rows or columns (**Structured**) to achieve real-world latency gains.